In [1]:
import tensorflow as tf
tf.__version__

'2.7.0'

In [4]:
import matplotlib
import matplotlib.pyplot as plt

from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

%matplotlib inline

In [5]:
!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz
!tar -xf ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz

--2021-11-19 09:18:42--  http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 142.250.207.80, 2404:6800:4005:81a::2010
Connecting to download.tensorflow.org (download.tensorflow.org)|142.250.207.80|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 244817203 (233M) [application/x-tar]
Saving to: ‘ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz’

ssd_resnet50_v1_fpn 100%[===================>] 233.48M  8.68MB/s    in 28s     

2021-11-19 09:19:11 (8.40 MB/s) - ‘ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz’ saved [244817203/244817203]



In [6]:
!mv ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/checkpoint ../models/research/object_detection/test_data/

In [7]:
tf.keras.backend.clear_session()

print('Building model and restoring weights for fine-tuning...', flush=True)
num_classes = 1
pipeline_config = '../models/research/object_detection/configs/tf2/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.config'
checkpoint_path = '../models/research/object_detection/test_data/checkpoint/ckpt-0'

# Load pipeline config and build a detection model.
#
# Since we are working off of a COCO architecture which predicts 90
# class slots by default, we override the `num_classes` field here to be just
# one (for our new rubber ducky class).
configs = config_util.get_configs_from_pipeline_file(pipeline_config)
model_config = configs['model']
model_config.ssd.num_classes = num_classes
model_config.ssd.freeze_batchnorm = True
detection_model = model_builder.build(
      model_config=model_config, is_training=True)

# Set up object-based checkpoint restore --- RetinaNet has two prediction
# `heads` --- one for classification, the other for box regression.  We will
# restore the box regression head but initialize the classification head
# from scratch (we show the omission below by commenting out the line that
# we would add if we wanted to restore both heads)
fake_box_predictor = tf.compat.v2.train.Checkpoint(
    _base_tower_layers_for_heads=detection_model._box_predictor._base_tower_layers_for_heads,
    # _prediction_heads=detection_model._box_predictor._prediction_heads,
    #    (i.e., the classification head that we *will not* restore)
    _box_prediction_head=detection_model._box_predictor._box_prediction_head,
    )
fake_model = tf.compat.v2.train.Checkpoint(
          _feature_extractor=detection_model._feature_extractor,
          _box_predictor=fake_box_predictor)
ckpt = tf.compat.v2.train.Checkpoint(model=fake_model)
ckpt.restore(checkpoint_path).expect_partial()

# Run model through a dummy image so that variables are created
image, shapes = detection_model.preprocess(tf.zeros([1, 640, 640, 3]))
prediction_dict = detection_model.predict(image, shapes)
_ = detection_model.postprocess(prediction_dict, shapes)
print('Weights restored!')

Building model and restoring weights for fine-tuning...


2021-11-19 09:21:39.399411: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:925] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2021-11-19 09:21:39.419329: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:925] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2021-11-19 09:21:39.419884: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:925] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2021-11-19 09:21:39.421033: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate

Weights restored!


In [9]:
_

{'detection_boxes': <tf.Tensor: shape=(1, 100, 4), dtype=float32, numpy=
 array([[[0.97750604, 0.9002061 , 0.99850345, 0.9686451 ],
         [0.9760343 , 0.02991995, 0.9975972 , 0.08242754],
         [0.975405  , 0.9139939 , 0.9979334 , 0.9888275 ],
         [0.97662854, 0.71680427, 0.9979408 , 0.7745187 ],
         [0.97669876, 0.69188493, 0.99795043, 0.7495679 ],
         [0.9767134 , 0.74158907, 0.9979533 , 0.79964113],
         [0.97681504, 0.6669103 , 0.9980069 , 0.72459054],
         [0.9768359 , 0.64187   , 0.9980208 , 0.6996589 ],
         [0.9768527 , 0.6168564 , 0.99803025, 0.6746605 ],
         [0.9768556 , 0.5918495 , 0.9980338 , 0.6496609 ],
         [0.9768534 , 0.41685066, 0.99803364, 0.4746599 ],
         [0.9768528 , 0.4793498 , 0.9980334 , 0.5371596 ],
         [0.9768528 , 0.50434977, 0.9980334 , 0.5621596 ],
         [0.9768528 , 0.45434982, 0.9980334 , 0.51215965],
         [0.9768528 , 0.52934974, 0.9980334 , 0.5871596 ],
         [0.97685266, 0.5543498 , 0.998033